In [1]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

import numpy as np
from time import time
import copy

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

CUDA = torch.cuda.is_available()

In [3]:
RAW_DATA_DIR = "../Data/Raw/"
PROCESSED_DATA_DIR = "../Data/Processed/Regression/"

class CovariateDataset(Dataset):
    def __init__(self, file_name_pattern, file_name_args, train_size=0.8, test_size=0.2, test_train_complement=True):
        self.train = True
        self.test_on_all = False
        
        self.file_name = file_name_pattern.format(*file_name_args, "covar")
        self.assignment_file_name = file_name_pattern.format(*file_name_args, "assignment")
        
        self.data = np.loadtxt(RAW_DATA_DIR + self.file_name + ".csv", delimiter=",")[:, 1:] # remove bias
        self.assignment_data = np.loadtxt(
            RAW_DATA_DIR + self.assignment_file_name + ".csv", delimiter=",").astype(int)
        
        self.all_indeces = np.array(range(len(self.data)))
        treat_indeces = self.all_indeces[self.assignment_data.astype(int) == 1]
        control_indeces = self.all_indeces[self.assignment_data.astype(int) == 0]
        
        num_training = int(len(self.data)*train_size)
        
        self.train_indeces = np.random.choice(self.all_indeces, num_training, replace=False)
        if test_train_complement:
            self.test_indeces = list(set(self.all_indeces)^set(self.train_indeces))      
        else:
            self.test_indeces = np.random.choice(self.all_indeces, int(len(self.data)*(1-test_size)), replace=False)
        
        num_treated_in_train = len(np.intersect1d(treat_indeces, self.train_indeces, assume_unique=True))
        num_control_in_train = num_training - num_treated_in_train
        
        treat_weight = num_training / (2 * num_treated_in_train)
        control_weight = num_training / (2 * num_control_in_train)
        
        weighter = np.vectorize(lambda index: treat_weight if index in\
            treat_indeces else control_weight)
        
        self.weights = weighter(self.all_indeces)
        
    def active_data(self, index=0):
        if self.train:
            return self.data[self.train_indeces], self.assignment_data[self.train_indeces], \
                self.weights[self.train_indeces][index]
        else:
            if self.test_on_all:
                indeces = self.all_indeces
            else: 
                indeces = self.test_indeces
            
            return self.data[indeces], self.assignment_data[indeces], 1
            
    def __getitem__(self, index):
        covar_data, assignment_data, weight_data = self.active_data(index)
        class_vector = np.zeros(2)
        class_vector[int(assignment_data[index])] = 1
        
        return (covar_data[index], class_vector, weight_data)

    def __len__(self):
        return self.active_data()[0].shape[0]
    
    def save_processed_data(self, data):
        name = PROCESSED_DATA_DIR + self.file_name+".csv"
        np.savetxt(name, data, delimiter=",")
        
def get_datasets(file_name_format, file_name_args, **kwargs):
    train_set = CovariateDataset(file_name_format, file_name_args, **kwargs)
    test_set = copy.deepcopy(train_set)
    test_set.train = False

    predict_set = copy.deepcopy(train_set)
    predict_set.train = False
    predict_set.test_on_all = True
    
    return train_set, test_set, predict_set

In [4]:
# Based on an example from https://github.com/pytorch/examples/blob/master/vae/main.py
# Extended to place a different prior on binary vs normal vars

class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        
        INTERMEDIATE_DIMS_1 = 16
        INTERMEDIATE_DIMS_2 = 16
        INTERMEDIATE_DIMS_3 = 16
        INTERMEDIATE_DIMS_4 = 16
#         INTERMEDIATE_DIMS_5 = 16
#         INTERMEDIATE_DIMS_6 = 8

        FEATURES = 10

        LOSS_SCALE = 1

        # ENCODER LAYERS
        self.dense1 = nn.Linear(FEATURES, INTERMEDIATE_DIMS_1)
        self.dense2 = nn.Linear(INTERMEDIATE_DIMS_1, INTERMEDIATE_DIMS_2)
        self.dense3 = nn.Linear(INTERMEDIATE_DIMS_2, INTERMEDIATE_DIMS_3)
        self.dense4 = nn.Linear(INTERMEDIATE_DIMS_3, INTERMEDIATE_DIMS_4)
#         self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, INTERMEDIATE_DIMS_5)
#         self.dense6 = nn.Linear(INTERMEDIATE_DIMS_5, INTERMEDIATE_DIMS_6)
        self.dense5 = nn.Linear(INTERMEDIATE_DIMS_4, 2)
        
        # Activations
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.6)

    def forward(self, x):
        h1 = self.dropout(self.relu(self.dense1(x)))
        h2 = self.dropout(self.relu(self.dense2(h1)))
        h3 = self.dropout(self.relu(self.dense3(h2)))
        h4 = self.dropout(self.relu(self.dense4(h3)))
#         h5 = self.dropout(self.relu(self.dense5(h4)))
#         h6 = self.dropout(self.relu(self.dense6(h5)))
        
        return self.softmax(self.dense5(h4))



def train(model, optimizer, epoch, train_loader, log_results=False):
    model.train()
    train_loss = 0

    for batch_idx, (data, target_class, weights) in enumerate(train_loader):
        data = Variable(data)
        target_class = Variable(target_class)
        weights = Variable(weights)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()
        
        optimizer.zero_grad()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)

        train_loss += loss.data[0]
        
        # Find the gradient and descend
        loss.backward()
        optimizer.step()
        
    if log_results:
        print('====> Epoch: {} Average loss: {:.8f}'.format(
              epoch, train_loss / len(train_loader.dataset)))
        
        
def test(model, epoch, test_loader):
    # toggle model to test / inference mode
    model.eval()
    test_loss = 0

    for i, (data, target_class, weights) in enumerate(test_loader):
        data = Variable(data, volatile=True)
        target_class = Variable(target_class, volatile=True)
        weights = Variable(weights, volatile=True)
        
        data = data.float()
        target_class = target_class.float()
        weights = weights.float()
        
        if CUDA:
            data = data.cuda()
            target_class = target_class.cuda()
            weights = weights.cuda()

        output_propensity = model(data)
        
        # calculate loss
        loss_criterion = nn.BCELoss(weight=weights.view(weights.shape[0], 1), size_average=False)
        loss = loss_criterion(output_propensity, target_class)
        test_loss += loss.data[0]

    test_loss /= len(test_loader.dataset)
    
    if CUDA:
        output_propensity = output_propensity.cpu()
        target_class = target_class.cpu()
        
    score = accuracy(output_propensity.data.numpy(), target_class.data.numpy(), verbose=False)
    print('====> Test set loss: {:.4f}, {}%'.format(test_loss, score*100))
    
def predict(model, predict_loader):
    # Show reconstruction
    model.eval()
    print("Training state: ", model.training)
    
    original_data, targets, _ = next(iter(predict_loader))
    
    original_data = Variable(original_data)
    original_data = original_data.float()
    
    if CUDA:
        original_data = original_data.cuda()
        
    return original_data, targets, model(original_data)

def accuracy(output_data, targets, verbose=True):
        
    classes = np.argmax(output_data, axis=1)
    targets = np.argmax(targets, axis=1)
    
    if verbose:
        print(classification_report(targets, classes))
    return accuracy_score(targets, classes)

In [5]:
def train_model(model_class, train_set, test_set, predict_set, dataset_number, verbose=True, model=None):
    if model is None:
        model = model_class()
        if CUDA:
            model = model.cuda()

    num_epochs = 750
    train_batch_size = 64
    test_batch_size = 250
    learning_rate = 1e-3
    lr_sched = True
         
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [int(num_epochs/5), int(num_epochs/2)], gamma=0.1)

    train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=test_batch_size, shuffle=True)
    predict_loader = DataLoader(predict_set, batch_size=1000, shuffle=False)
    
    for epoch in range(1, num_epochs+1):
        checkpoint_interval = int(num_epochs/10)
        
        if lr_sched:
            scheduler.step()

        log = False
        if epoch%checkpoint_interval == 0:
            log = True
            
        train(model, optimizer, epoch, train_loader, log_results=log)
        if log:
            test(model, epoch, test_loader)
    
    original_data, targets, output = predict(model, predict_loader)
    if CUDA:
        output = output.cpu()
        targets = targets.cpu()
    
    return model, original_data, targets, output

def encode_data(dataset, output_data):
    
    if CUDA:
        output_data = output_data.cpu()
        
    dataset.save_processed_data(output_data.data.numpy()[:, 1])


In [7]:
def run_logistic(train_set, verbose=True):
    model = LogisticRegression(class_weight="balanced")
    
    X = train_set.data
    y = train_set.assignment_data

    X_train = X[train_set.train_indeces]
    X_test = X[train_set.test_indeces]
    y_train = y[train_set.train_indeces]
    y_test = y[train_set.test_indeces]

    model.fit(X_train, y_train)
    predictions = model.predict(X)
    
    if verbose:
        print(classification_report(y, predictions))
    
    return accuracy_score(y, predictions)

In [8]:
train_set, test_set, predict_set = get_datasets(
    "n_{}_model_{}_v_{}_{}_data", [1000, "G_mod_nadd_mod_nlin", 1],
    train_size=0.8, test_train_complement=True)

start = time()
trained_model, original_data, targets, output = \
    train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
print("Elapsed: ", time() - start)


acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
print("Complete set accuracy: {}%".format(acc*100))

encode_data(train_set, output)

====> Epoch: 75 Average loss: 1.32145905
====> Test set loss: 1.2942, 62.0%
====> Epoch: 150 Average loss: 1.25285402
====> Test set loss: 1.2176, 68.5%
====> Epoch: 225 Average loss: 1.28917585
====> Test set loss: 1.2106, 68.5%
====> Epoch: 300 Average loss: 1.23299732
====> Test set loss: 1.2073, 68.0%
====> Epoch: 375 Average loss: 1.24430372
====> Test set loss: 1.1989, 68.0%
====> Epoch: 450 Average loss: 1.23220974
====> Test set loss: 1.1982, 68.0%
====> Epoch: 525 Average loss: 1.26749196
====> Test set loss: 1.1987, 68.0%
====> Epoch: 600 Average loss: 1.24656528
====> Test set loss: 1.1988, 68.0%
====> Epoch: 675 Average loss: 1.25521823
====> Test set loss: 1.2001, 68.0%
====> Epoch: 750 Average loss: 1.26520532
====> Test set loss: 1.1999, 68.0%
Training state:  False
Elapsed:  26.38823103904724
Complete set accuracy: 68.7%


### Runner

In [12]:
assignment_model_names = ['A_add_lin', 'B_add_mild_nlin', 'C_add_mod_nlin', 'D_mild_nadd_lin',
                     'E_mild_nadd_mild_nlin', 'F_mod_nadd_lin', 'G_mod_nadd_mod_nlin']

nn_accuracies = []
log_accuracies = []

for dataset_number in range(275, 300):
    print("Starting run for Dataset {}".format(dataset_number))
    
    for model_name in assignment_model_names:
        print("---- Running for model name: ", model_name)
        
        start = time()

        train_set, test_set, predict_set = get_datasets(
            "n_{}_model_{}_v_{}_{}_data", [1000, model_name, dataset_number], train_size=0.8)

        trained_model, original_data, targets, output = \
            train_model(Regressor, train_set, test_set,predict_set, 1,verbose=True)
        
        nn_acc = accuracy(output.data.cpu().numpy(), targets.numpy(), verbose=False)
        print("Complete set accuracy: {}%".format(nn_acc*100))
        
        log_acc = run_logistic(train_set, verbose=False)
        print("Log accuracy: {}%".format(log_acc*100))
        
        nn_accuracies.append(nn_acc)
        log_accuracies.append(log_acc)

        encode_data(train_set, output)

        print("---- Done in ", time() - start, " seconds\n")
                
    print("================\n\n")

Starting run for Dataset 250
---- Running for model name:  A_add_lin
====> Epoch: 75 Average loss: 1.21334507
====> Test set loss: 1.2038, 71.0%
====> Epoch: 150 Average loss: 1.20678432
====> Test set loss: 1.1966, 69.0%
====> Epoch: 225 Average loss: 1.18914899
====> Test set loss: 1.1904, 69.0%
====> Epoch: 300 Average loss: 1.20496358
====> Test set loss: 1.1909, 69.0%
====> Epoch: 375 Average loss: 1.15847391
====> Test set loss: 1.1900, 69.0%
====> Epoch: 450 Average loss: 1.17403279
====> Test set loss: 1.1909, 69.0%
====> Epoch: 525 Average loss: 1.15826491
====> Test set loss: 1.1909, 69.0%
====> Epoch: 600 Average loss: 1.16790349
====> Test set loss: 1.1909, 69.0%
====> Epoch: 675 Average loss: 1.13954620
====> Test set loss: 1.1907, 69.0%
====> Epoch: 750 Average loss: 1.20025498
====> Test set loss: 1.1909, 69.0%
Training state:  False
Complete set accuracy: 74.8%
Log accuracy: 72.89999999999999%
---- Done in  29.250168085098267  seconds

---- Running for model name:  B_ad

====> Epoch: 675 Average loss: 1.17592195
====> Test set loss: 1.2315, 63.0%
====> Epoch: 750 Average loss: 1.16707004
====> Test set loss: 1.2319, 62.5%
Training state:  False
Complete set accuracy: 70.7%
Log accuracy: 69.3%
---- Done in  55.517998933792114  seconds

---- Running for model name:  C_add_mod_nlin
====> Epoch: 75 Average loss: 1.31085980
====> Test set loss: 1.2300, 68.5%
====> Epoch: 150 Average loss: 1.23403026
====> Test set loss: 1.1423, 70.5%
====> Epoch: 225 Average loss: 1.24034625
====> Test set loss: 1.1362, 70.5%
====> Epoch: 300 Average loss: 1.20161272
====> Test set loss: 1.1289, 71.0%
====> Epoch: 375 Average loss: 1.22348391
====> Test set loss: 1.1259, 70.5%
====> Epoch: 450 Average loss: 1.22668410
====> Test set loss: 1.1251, 70.5%
====> Epoch: 525 Average loss: 1.22486974
====> Test set loss: 1.1251, 70.5%
====> Epoch: 600 Average loss: 1.23893248
====> Test set loss: 1.1249, 70.5%
====> Epoch: 675 Average loss: 1.23897053
====> Test set loss: 1.1248, 

====> Epoch: 450 Average loss: 1.15084219
====> Test set loss: 1.0638, 76.0%
====> Epoch: 525 Average loss: 1.10616369
====> Test set loss: 1.0634, 76.0%
====> Epoch: 600 Average loss: 1.11547925
====> Test set loss: 1.0630, 76.0%
====> Epoch: 675 Average loss: 1.11763217
====> Test set loss: 1.0635, 76.0%
====> Epoch: 750 Average loss: 1.20306067
====> Test set loss: 1.0632, 76.0%
Training state:  False
Complete set accuracy: 79.3%
Log accuracy: 76.8%
---- Done in  56.87483501434326  seconds

---- Running for model name:  E_mild_nadd_mild_nlin
====> Epoch: 75 Average loss: 1.25825727
====> Test set loss: 1.2626, 65.0%
====> Epoch: 150 Average loss: 1.19287161
====> Test set loss: 1.2274, 66.0%
====> Epoch: 225 Average loss: 1.19884831
====> Test set loss: 1.2244, 66.0%
====> Epoch: 300 Average loss: 1.19378633
====> Test set loss: 1.2212, 65.5%
====> Epoch: 375 Average loss: 1.16617222
====> Test set loss: 1.2203, 64.5%
====> Epoch: 450 Average loss: 1.18145055
====> Test set loss: 1.

====> Epoch: 225 Average loss: 1.22541291
====> Test set loss: 1.1171, 75.0%
====> Epoch: 300 Average loss: 1.16515118
====> Test set loss: 1.1192, 75.0%
====> Epoch: 375 Average loss: 1.21142216
====> Test set loss: 1.1162, 76.0%
====> Epoch: 450 Average loss: 1.16210284
====> Test set loss: 1.1162, 75.5%
====> Epoch: 525 Average loss: 1.19023780
====> Test set loss: 1.1161, 75.5%
====> Epoch: 600 Average loss: 1.15633847
====> Test set loss: 1.1158, 75.5%
====> Epoch: 675 Average loss: 1.19210158
====> Test set loss: 1.1154, 75.5%
====> Epoch: 750 Average loss: 1.15000021
====> Test set loss: 1.1152, 76.0%
Training state:  False
Complete set accuracy: 77.4%
Log accuracy: 72.1%
---- Done in  55.30870604515076  seconds

---- Running for model name:  G_mod_nadd_mod_nlin
====> Epoch: 75 Average loss: 1.30373453
====> Test set loss: 1.2527, 64.5%
====> Epoch: 150 Average loss: 1.21289224
====> Test set loss: 1.1745, 67.0%
====> Epoch: 225 Average loss: 1.22818933
====> Test set loss: 1.16

====> Epoch: 75 Average loss: 1.26764108
====> Test set loss: 1.2041, 69.0%
====> Epoch: 150 Average loss: 1.22512663
====> Test set loss: 1.1678, 68.0%
====> Epoch: 225 Average loss: 1.19136334
====> Test set loss: 1.1732, 68.0%
====> Epoch: 300 Average loss: 1.20945621
====> Test set loss: 1.1740, 68.0%
====> Epoch: 375 Average loss: 1.21451942
====> Test set loss: 1.1772, 67.5%
====> Epoch: 450 Average loss: 1.18415457
====> Test set loss: 1.1761, 67.0%
====> Epoch: 525 Average loss: 1.22148257
====> Test set loss: 1.1757, 67.0%
====> Epoch: 600 Average loss: 1.15207600
====> Test set loss: 1.1758, 67.0%
====> Epoch: 675 Average loss: 1.22382368
====> Test set loss: 1.1756, 67.0%
====> Epoch: 750 Average loss: 1.19959745
====> Test set loss: 1.1752, 67.0%
Training state:  False
Complete set accuracy: 69.1%
Log accuracy: 70.0%
---- Done in  55.17376399040222  seconds

---- Running for model name:  B_add_mild_nlin
====> Epoch: 75 Average loss: 1.32088841
====> Test set loss: 1.3046, 6

...

====> Epoch: 75 Average loss: 1.31467990
====> Test set loss: 1.2500, 70.0%
====> Epoch: 150 Average loss: 1.23734236
====> Test set loss: 1.1451, 71.0%
====> Epoch: 225 Average loss: 1.23087464
====> Test set loss: 1.1373, 71.5%
====> Epoch: 300 Average loss: 1.25372636
====> Test set loss: 1.1292, 71.5%
====> Epoch: 375 Average loss: 1.21871305
====> Test set loss: 1.1243, 71.5%
====> Epoch: 450 Average loss: 1.21589152
====> Test set loss: 1.1246, 71.5%
====> Epoch: 525 Average loss: 1.25585916
====> Test set loss: 1.1247, 71.5%
====> Epoch: 600 Average loss: 1.23089632
====> Test set loss: 1.1245, 71.5%
====> Epoch: 675 Average loss: 1.25905030
====> Test set loss: 1.1245, 71.5%
====> Epoch: 750 Average loss: 1.20859244
====> Test set loss: 1.1239, 71.5%
Training state:  False
Complete set accuracy: 73.0%
Log accuracy: 67.7%
---- Done in  66.67315721511841  seconds

---- Running for model name:  D_mild_nadd_lin
====> Epoch: 75 Average loss: 1.21816752
====> Test set loss: 1.0534, 7

====> Epoch: 75 Average loss: 1.19057410
====> Test set loss: 1.0986, 76.5%
====> Epoch: 150 Average loss: 1.17433410
====> Test set loss: 1.0904, 73.0%
====> Epoch: 225 Average loss: 1.14185076
====> Test set loss: 1.0904, 71.5%
====> Epoch: 300 Average loss: 1.13340048
====> Test set loss: 1.0950, 70.5%
====> Epoch: 375 Average loss: 1.15836723
====> Test set loss: 1.0958, 70.0%
====> Epoch: 450 Average loss: 1.11576487
====> Test set loss: 1.0960, 70.0%
====> Epoch: 525 Average loss: 1.12259052
====> Test set loss: 1.0959, 70.0%
====> Epoch: 600 Average loss: 1.15480227
====> Test set loss: 1.0965, 70.0%
====> Epoch: 675 Average loss: 1.13681136
====> Test set loss: 1.0969, 70.0%
====> Epoch: 750 Average loss: 1.12079443
====> Test set loss: 1.0966, 70.0%
Training state:  False
Complete set accuracy: 74.6%
Log accuracy: 74.5%
---- Done in  69.2667589187622  seconds

---- Running for model name:  F_mod_nadd_lin
====> Epoch: 75 Average loss: 1.31404070
====> Test set loss: 1.3030, 68.

====> Epoch: 75 Average loss: 1.25463388
====> Test set loss: 1.2032, 62.5%
====> Epoch: 150 Average loss: 1.23084403
====> Test set loss: 1.1564, 70.0%
====> Epoch: 225 Average loss: 1.21282888
====> Test set loss: 1.1530, 70.0%
====> Epoch: 300 Average loss: 1.20558853
====> Test set loss: 1.1525, 70.0%
====> Epoch: 375 Average loss: 1.16872850
====> Test set loss: 1.1531, 69.0%
====> Epoch: 450 Average loss: 1.19326830
====> Test set loss: 1.1506, 69.5%
====> Epoch: 525 Average loss: 1.19618567
====> Test set loss: 1.1502, 69.5%
====> Epoch: 600 Average loss: 1.16062933
====> Test set loss: 1.1491, 70.5%
====> Epoch: 675 Average loss: 1.13381235
====> Test set loss: 1.1486, 70.5%
====> Epoch: 750 Average loss: 1.19925678
====> Test set loss: 1.1491, 70.0%
Training state:  False
Complete set accuracy: 75.0%
Log accuracy: 66.3%
---- Done in  61.6259880065918  seconds



Starting run for Dataset 274
---- Running for model name:  A_add_lin
====> Epoch: 75 Average loss: 1.22895282
====> 

====> Epoch: 750 Average loss: 1.17187872
====> Test set loss: 1.1181, 74.5%
Training state:  False
Complete set accuracy: 75.9%
Log accuracy: 72.8%
---- Done in  76.08613467216492  seconds

---- Running for model name:  B_add_mild_nlin
====> Epoch: 75 Average loss: 1.23207791
====> Test set loss: 1.2139, 70.0%
====> Epoch: 150 Average loss: 1.18716567
====> Test set loss: 1.1714, 71.0%
====> Epoch: 225 Average loss: 1.12380584
====> Test set loss: 1.1771, 70.5%
====> Epoch: 300 Average loss: 1.17446727
====> Test set loss: 1.1777, 71.0%
====> Epoch: 375 Average loss: 1.12737920
====> Test set loss: 1.1761, 71.0%
====> Epoch: 450 Average loss: 1.14434025
====> Test set loss: 1.1768, 71.0%
====> Epoch: 525 Average loss: 1.12190791
====> Test set loss: 1.1765, 71.0%
====> Epoch: 600 Average loss: 1.12095000
====> Test set loss: 1.1765, 71.0%
====> Epoch: 675 Average loss: 1.14635811
====> Test set loss: 1.1769, 71.0%
====> Epoch: 750 Average loss: 1.16046999
====> Test set loss: 1.1770, 

KeyboardInterrupt: 